<a href="https://colab.research.google.com/github/puneethkumarvydya/Capstone-Project/blob/main/Capstone_Project_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>